## imports + setup

In [1]:
import os
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate
import json
from pydantic import BaseModel, Field
import string
import re
from typing import Optional 

In [2]:
try:
    with open("../secrets.json") as f:
        secrets = json.load(f)
    open_ai_key = secrets["openai"]
    anthropic_key = secrets["anthropic"]
    os.environ["OPENAI_API_KEY"] = open_ai_key
    os.environ["ANTHROPIC_API_KEY"] = anthropic_key
    print("API key loaded.")
except FileNotFoundError:
    print("Secrets file not found. YOU NEED THEM TO RUN THIS.")

API key loaded.


In [3]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)
llm2 = ChatAnthropic(model_name="claude-3-5-sonnet-20240620", temperature=0.1, max_tokens_to_sample=2048, api_key=os.getenv("ANTHROPIC_API_KEY"))

## brainstorm story ideas

In [86]:
class StoryIdea(BaseModel):
    """Idea for a story."""
    title: str = Field(description="Title of story")
    summary: str = Field(description="2-3 sentence summary of story")

class StoryIdeaList(BaseModel):
    """List of story ideas."""
    ideas: list[StoryIdea] = Field(description="List of story ideas")

idea_llm = llm.with_structured_output(StoryIdeaList)

genre = "magical realism"
num_ideas = 10
idea_prompt = f"please brainstorm {num_ideas} idea for a {genre} novel"

In [87]:
invalid_input = True
while invalid_input: # try until valid input
    # see if exception is thrown
    try:
        res = idea_llm.invoke(idea_prompt)
        invalid_input = False
    except Exception as e:
        print("Exception thrown. Trying again.")
        print("Error:", e)
        invalid_input = True

In [88]:
idealist = []
for idea in res.ideas:
    idealist.append({"title": idea.title, "summary": idea.summary})

idealist

[{'title': 'The Whispering Trees',
  'summary': "In a small town where the trees can communicate with the residents, a young girl discovers that they hold the memories of the town's past. As she listens to their stories, she uncovers a hidden truth about her family's history that changes everything."},
 {'title': "The Clockmaker's Gift",
  'summary': 'An aging clockmaker creates timepieces that can manipulate time in small ways. When a young man stumbles upon one of his clocks, he learns to relive moments from his past, but each use comes with unforeseen consequences.'},
 {'title': "The Dreamweaver's Daughter",
  'summary': "In a world where dreams can be woven into reality, a girl inherits her mother's ability to craft dreams. As she navigates her newfound power, she must confront the dark dreams that threaten to consume her town."},
 {'title': 'The Library of Forgotten Stories',
  'summary': 'A mysterious library appears overnight in a bustling city, containing books that tell the st

In [89]:
selected_idea = 'The Color Thief'
# get the selected idea
selected_idea = [idea for idea in idealist if idea["title"] == selected_idea][0]

selected_idea

{'title': 'The Color Thief',
 'summary': 'In a monochrome world, a young artist discovers a hidden realm where colors are stolen and trapped. With the help of a mischievous spirit, she embarks on a quest to restore color to her world and uncover the truth behind the theft.'}

In [90]:
story_name = "color"
if not os.path.exists(f"json/{story_name}"):
    os.makedirs(f"json/{story_name}")

In [91]:
# save the idea to a file
with open(f"json/{story_name}/idea.json", "w") as f:
    json.dump(selected_idea, f, indent=4)

## generate chapter outline

In [92]:
# read in idea from file
with open(f"json/{story_name}/idea.json") as f:
    idea = json.load(f)

In [93]:
title = idea["title"]
summary = idea["summary"]

In [94]:
# create chapter outline
class Character(BaseModel):
    """Character in a story."""
    name: str = Field(description="Name of character")
    description: str = Field(description="Description/role of character in this chapter")
    # group: str = Field(description="Group that this character belongs to (e.g., main characters, upper class, talking animals). There should be at least 2 different groups of characters.")

class Location(BaseModel):
    """Location in a story."""
    name: str = Field(description="Name of location")
    description: str = Field(description="Description/role of location in this chapter")

class Chapter(BaseModel):
    """Chapter in a story."""
    chapter: str = Field(description="Title of chapter, e.g. 'Chapter 1: The Beginning'")
    summary: str = Field(description="1 line summary of chapter")
    description: str = Field(description="longer 2-3 sentence summary of chapter")
    importance: float = Field(description="Importance of chapter to story relative to other chapters from 0 to 1 (0 = not important, 1 = very important)")
    conflict: float = Field(description="Conflict level of chapter from 0 to 1 (0 = no conflict, 1 = high conflict)")
    # characters: list[Character] = Field(description="List of characters in chapter.")
    # locations: list[Location] = Field(description="List of locations in chapter.")

class ChapterList(BaseModel):
    """List of chapters."""
    ideas: list[Chapter] = Field(description="List of chapters")

chapter_llm = llm.with_structured_output(ChapterList)

num_chapters = 12
num_chars = 20
num_locs = 8

chapter_prompt=f"Please outline {num_chapters} chapters for a novel titled '{title}' with the summary: '{summary}'. There should be at least {num_chars} unique characters in the story and at least {num_locs} unique locations."

In [95]:
invalid_input = True
while invalid_input: # try until valid input
    # see if exception is thrown
    try:
        res = chapter_llm.invoke(chapter_prompt)
        invalid_input = False
    except Exception as e:
        print("Exception thrown. Trying again.")
        print("Error:", e)
        invalid_input = True

In [96]:
# convert to json
chapterlist = []
for chapter in res.ideas:
    # chapterlist.append({"chapter": chapter.chapter, "summary": chapter.summary, "description": chapter.description, "importance": chapter.importance, "conflict": chapter.conflict, "characters": [{"name": c.name, "description": c.description} for c in chapter.characters], "locations": [{"name": l.name, "description": l.description} for l in chapter.locations]})
    chapterlist.append({"chapter": chapter.chapter, "summary": chapter.summary, "description": chapter.description, "importance": chapter.importance, "conflict": chapter.conflict})

len(chapterlist), chapterlist

(12,
 [{'chapter': 'Chapter 1: The Gray Awakening',
   'summary': 'In a world devoid of color, a young artist named Lila struggles to find inspiration.',
   'description': 'Lila wakes up in her monochrome town, feeling the weight of the dullness around her. She reflects on her dreams of color and beauty, but her surroundings stifle her creativity. This chapter introduces Lila and her longing for a vibrant world, setting the stage for her journey.',
   'importance': 0.9,
   'conflict': 0.3},
  {'chapter': 'Chapter 2: The Mysterious Encounter',
   'summary': 'Lila meets a mischievous spirit named Zephyr who reveals the existence of a hidden realm.',
   'description': "While wandering through the gray streets, Lila encounters Zephyr, a playful spirit who can manipulate shadows. Zephyr tells her about a hidden realm where colors are stolen and trapped, igniting Lila's curiosity and desire to explore beyond her monochrome life.",
   'importance': 0.8,
   'conflict': 0.4},
  {'chapter': 'Cha

In [97]:
# save the chapter outline to a file
with open(f"json/{story_name}/chapters.json", "w") as f:
    json.dump(chapterlist, f, indent=4)

## generate scene outline

In [98]:
# read in chapter outline from file
with open(f"json/{story_name}/chapters.json") as f:
    chapters = json.load(f)

In [99]:
# fill in scene details
class SceneCharacter(BaseModel):
    """Character in a scene."""
    name: str = Field(description="Name of character")
    role: str = Field(description="1 line summary of role of character in this scene")
    importance: float = Field(description="Importance of character in scene from 0 to 1 (0 = not important, 1 = very important)")
    emotion: str = Field(description="Emotion of character in scene, described in a few words or a short phrase")
    sentiment: float = Field(description="Sentiment of character in scene from -1 to 1 (-1 = negative, 0 = neutral, 1 = positive)")

class Scene(BaseModel):
    """Scene in a story."""
    title: str = Field(description="Name of scene")
    summary: str = Field(description="1 line summary of scene")
    location: str = Field(description="Location of scene")
    importance: float = Field(description="Importance of scene in chapter from 0 to 1 (0 = not important, 1 = very important)")
    conflict: float = Field(description="Conflict level of scene from 0 to 1 (0 = no conflict, 1 = high conflict)")
    characters: list[SceneCharacter] = Field(description="List of characters in scene")

class SceneList(BaseModel):
    """List of scenes."""
    scenes: list[Scene] = Field(description="List of scenes in this chapter")

scene_llm = llm.with_structured_output(SceneList)

In [100]:
chapter_scenes = []
for i, chapter in enumerate(chapters):
    print(f"{chapter['chapter']}")
    invalid_input = True
    while invalid_input:
        try:
            prompt = f"You are a novelist writing a novel titled '{title}' about: '{summary}'. Please list all the key scenes that would be in this chapter: {chapter}."
            res = scene_llm.invoke(prompt)
            chapter_scenes.append(res.scenes)
            invalid_input = False
        except Exception as e:
            print("Exception thrown. Trying again.")
            print("Error:", e)
            invalid_input = True
    # break

Chapter 1: The Gray Awakening
Chapter 2: The Mysterious Encounter
Chapter 3: The Portal to Color
Chapter 4: The Color Keepers
Chapter 5: The Thief's Lair
Chapter 6: Allies and Enemies
Chapter 7: The Battle for Color
Chapter 8: The Heart of the Realm
Chapter 9: Betrayal and Sacrifice
Chapter 10: The Final Confrontation
Chapter 11: Restoration of Color
Chapter 12: A New Beginning


In [101]:
all_scenes = []
for i, chapter in enumerate(chapter_scenes):
    for j, scene in enumerate(chapter):
        new_scene = {}
        new_scene["title"] = scene.title
        new_scene["number"] = j + 1
        new_scene["summary"] = scene.summary
        new_scene["chapter"] = chapters[i]["chapter"]
        new_scene["location"] = scene.location
        new_scene["importance"] = scene.importance
        new_scene["conflict"] = scene.conflict
        new_scene["characters"] = [{"name": c.name, "role": c.role, "importance": c.importance, "emotion": c.emotion, "sentiment": c.sentiment} for c in scene.characters]
        all_scenes.append(new_scene)

len(all_scenes), all_scenes

(48,
 [{'title': "Lila's Awakening",
   'number': 1,
   'summary': 'Lila wakes up in her gray bedroom, surrounded by monochrome art supplies.',
   'chapter': 'Chapter 1: The Gray Awakening',
   'location': "Lila's bedroom",
   'importance': 0.9,
   'conflict': 0.2,
   'characters': [{'name': 'Lila',
     'role': 'Protagonist and aspiring artist',
     'importance': 1.0,
     'emotion': 'Frustrated and longing',
     'sentiment': -0.5}]},
  {'title': 'The Gray Town',
   'number': 2,
   'summary': 'Lila walks through her dull town, observing the lack of color in her surroundings.',
   'chapter': 'Chapter 1: The Gray Awakening',
   'location': 'Monochrome town square',
   'importance': 0.8,
   'conflict': 0.3,
   'characters': [{'name': 'Lila',
     'role': 'Protagonist reflecting on her environment',
     'importance': 1.0,
     'emotion': 'Melancholic',
     'sentiment': -0.6}]},
  {'title': 'Dreams of Color',
   'number': 3,
   'summary': 'Lila daydreams about a world filled with color

In [102]:
# save the scene details to a file
with open(f"json/{story_name}/scenes.json", "w") as f:
    json.dump(all_scenes, f, indent=4)

## get character/location lists

In [103]:
# read in scene details from file
with open(f"json/{story_name}/scenes.json") as f:
    scenes = json.load(f)

In [104]:
# list all unique characters
characters = {}
for scene in scenes:
    for character in scene["characters"]:
        if character["name"] not in characters:
            characters[character["name"]] = character["role"]

len(characters), characters

(20,
 {'Lila': 'Protagonist and aspiring artist',
  'Passersby': 'Background characters illustrating the dullness of the world',
  'Zephyr': 'Mischievous spirit guiding Lila',
  'Mischievous Spirit': 'Guide introducing Lila to the Keepers',
  'Red Keeper': 'Guardian of the color red',
  'Blue Keeper': 'Guardian of the color blue',
  'Yellow Keeper': 'Guardian of the color yellow',
  'Green Keeper': 'Guardian of the color green',
  'Malachai': 'The dark sorcerer revealing his intentions to drain color from the world.',
  'Sage': 'The wise sage providing guidance',
  'Finn': 'The cunning trickster with a playful attitude',
  'Brave Warrior': 'The strong ally ready to fight',
  'Allies': 'Various friends joining Lila',
  "Malachai's Minions": "The antagonistic forces attacking Lila's group",
  'Guardian of Colors': 'Protector of the Heart of the Realm',
  'Milo': 'The traitor, conflicted about his actions',
  'Sera': "Lila's close friend, trying to mediate",
  'The Spirit': "Lila's guide 

In [105]:
# list all unique locations
locations = []
for scene in scenes:
    if scene["location"] not in locations:
        locations.append(scene["location"])

len(locations), locations

(35,
 ["Lila's bedroom",
  'Monochrome town square',
  'Park bench in the town',
  'Art studio',
  "Lila's dreamscape",
  'Monochrome Town',
  'Abandoned Art Gallery',
  'Inside the Portal',
  'Hidden Realm of Colors',
  "Color Keepers' Sanctuary",
  'Sanctuary Garden',
  'Council Chamber',
  "Malachai's Lair",
  'Hall of Stolen Colors',
  "Malachai's Throne Room",
  'Throne Room',
  "Exit of Malachai's Lair",
  'Vibrant Glade',
  'The Hidden Realm',
  'The outskirts of the battlefield',
  'The Battlefield',
  'The Heart of the Battlefield',
  'Heart of the Realm',
  'The hidden base of the group',
  "Lila's hideout",
  "The Color Thief's lair",
  "Lila's hometown",
  'The battlefield, now quiet and still',
  'The heart of the hidden realm',
  'The once monochrome landscape now bursting with color',
  "The portal connecting the hidden realm to Lila's world",
  "Lila's studio, now filled with vibrant colors",
  "Lila's art studio",
  'Community center',
  "Lila's favorite spot in town"]

first clean up duplicate characters/locations

In [106]:
dup_prompt = """
You are given a list of characters with some duplicates or alternate names.
Your task is to group all the alternate names for each unique character under a single full name, using the descriptions as guidance.
Don't keep characters like "Narrator" unless they are never referred to by another name.
Separate characters that are related to others (e.g., "Bob" and "Ms. Doe (Bob's mom)" and "Bob's sister") into separate entries.

First create a new character list by removing all duplicates and alternate names and only keeping the full name for each character.

Then, output a JSON dictionary where the key is each character in your new list and the value is a list of all possible alternate names. 
All names should be taken directly from the original character list and should only appear once in the output.

For example:
{{
    "Bob Smith": ["Bob", "Bobby", "Robert"],
    "Jane Doe": ["Jane", "Ms. Doe", "Bob's mother"]
}}
"""

location_dup_prompt = """
You are given a list of locations with some duplicates or alternate names.
Your task is to group all the alternate or similiar names for each unique location under a single location name.

First create a new location list by removing all duplicates and alternate names and only keeping the full name for each location.
If one location is inside another or nearby to it, only keep the larger location (e.g., "Bob's bedroom", "Bob's garden", and "Bob Smith's castle" can be listed under "Bob's house").
Then, output a JSON dictionary where the key is each location in your new list and the value is a list of all possible alternate location names.
All locations should be taken directly from the original location list and should only appear once in the output dict.

For example:
{{
    "Los Angeles": ["LA", "City of Angels", "Cafe in Los Angeles"],
    "Bob's house": ["Bob's bedroom", "Bob Smith's garden", "Bob's castle"]
}}
"""

# if using llm2 (claude)
dup_prompt += "Just output the JSON dictionary as the final result without any additional information."
location_dup_prompt += "Just output the JSON dictionary as the final result without any additional information."

prompt_template = ChatPromptTemplate.from_messages([("system", dup_prompt), ("human", "{input}")])
location_prompt_template = ChatPromptTemplate.from_messages([("system", location_dup_prompt), ("human", "{input}")])

In [107]:
# characters
final_prompt = prompt_template.invoke(f"character list: {characters}")
character_llm = llm2
output = character_llm.invoke(final_prompt)
output_formatted = output.content
output = json.loads(output_formatted)

In [108]:
# locations
final_location_prompt = location_prompt_template.invoke(f"location list: {locations}")
location_llm = llm2
location_output = location_llm.invoke(final_location_prompt)
location_output_formatted = location_output.content
location_output = json.loads(location_output_formatted)

In [109]:
# Function to find the correct key based on a search string, ensuring whole-word matches
def find_key_from_alias(text, character_dict):
    for key, aliases in character_dict.items():
        for alias in sorted(aliases, key=len, reverse=True):
            # Match the alias as a whole word, and ensure no partial matches within other words
            pattern = rf'\b{re.escape(alias)}\b(?!\'s)'  # (?!\'s) ensures no match for possessives like Mitsuha's
            if re.search(pattern, text):
                return key  # Return the key if the alias is found as a whole word
    return text  # Return the original text if no match is found

# Load the scenes
with open(f"json/{story_name}/scenes.json") as f:
    scenes = json.load(f)

# Capitalize output and location output
capitalized_output = {string.capwords(key): [string.capwords(name) for name in value] for key, value in output.items()}
capitalized_location_output = {string.capwords(key): [string.capwords(name) for name in value] for key, value in location_output.items()}

# Process each scene
for scene in scenes:
    # Process characters
    for character in scene["characters"]:
        char = string.capwords(character["name"])
        found_char = False

        # Check if the character's name is a key in the dictionary
        if char in capitalized_output:
            found_char = True
            character["name"] = char  # Capitalize the character name
        
        # If not found in the dictionary, check the aliases
        if not found_char:
            # Find the right key for the alias (if exists)
            matched_key = find_key_from_alias(char, capitalized_output)
            if matched_key != char:
                character["name"] = matched_key
                found_char = True
        if not found_char:
            character["name"] = char

    # Process location
    loc = string.capwords(scene["location"])
    found_loc = False

    # Check if the location is a key in the location dictionary
    if loc in capitalized_location_output:
        found_loc = True
        scene["location"] = loc  # Capitalize the location

    # If not found in the dictionary, check the aliases
    if not found_loc:
        matched_key = find_key_from_alias(loc, capitalized_location_output)
        if matched_key != loc:
            scene["location"] = matched_key
            found_loc = True
    if not found_loc:
        scene["location"] = loc

# Save the updated scenes
with open(f"json/{story_name}/scenes.json", "w") as f:
    json.dump(scenes, f, indent=4)

In [110]:
# count characters and locations again
# list all unique characters
characters = {}
for scene in scenes:
    for character in scene["characters"]:
        if character["name"] not in characters:
            characters[character["name"]] = character["role"]

len(characters), characters

(18,
 {'Lila': 'Protagonist and aspiring artist',
  'Passersby': 'Background characters illustrating the dullness of the world',
  'Zephyr': 'Mischievous spirit guiding Lila',
  'Red Keeper': 'Guardian of the color red',
  'Blue Keeper': 'Guardian of the color blue',
  'Yellow Keeper': 'Guardian of the color yellow',
  'Green Keeper': 'Guardian of the color green',
  'Malachai': 'The dark sorcerer revealing his intentions to drain color from the world.',
  'Sage': 'The wise sage providing guidance',
  'Finn': 'The cunning trickster with a playful attitude',
  'Brave Warrior': 'The strong ally ready to fight',
  'Allies': 'Various friends joining Lila',
  "Malachai's Minions": "The antagonistic forces attacking Lila's group",
  'Guardian Of Colors': 'Protector of the Heart of the Realm',
  'Milo': 'The traitor, conflicted about his actions',
  'Sera': "Lila's close friend, trying to mediate",
  'Color Keepers': 'Allies who support Lila in the restoration process.',
  'Townspeople': "Aud

In [111]:
# list all unique locations
locations = []
for scene in scenes:
    if scene["location"] not in locations:
        locations.append(scene["location"])

len(locations), locations

(12,
 ["Lila's House",
  'Monochrome Town',
  'Art Studio',
  "Lila's Dreamscape",
  'Abandoned Art Gallery',
  'Hidden Realm Of Colors',
  "Color Keepers' Sanctuary",
  "Malachai's Lair",
  'Vibrant Glade',
  'The Battlefield',
  'The Hidden Base Of The Group',
  'The Once Monochrome Landscape Now Bursting With Color'])

now fill in character details

In [112]:
# fill in character details
class LegendCharacter(BaseModel):
    """Character in a story."""
    name: str = Field(description="Name of character")
    description: str = Field(description="Description/role of character in this story")
    group: str = Field(description="Group that this character belongs to (e.g., main characters, upper class, talking animals). There should be at least 2 different groups of characters.")

class LegendCharacterList(BaseModel):
    """List of characters."""
    characters: list[LegendCharacter] = Field(description="List of characters. Only include characters from the original list of characters.")

legendchar_llm = llm.with_structured_output(LegendCharacterList)

# fill in location details
class LegendLocation(BaseModel):
    """Location in a story."""
    name: str = Field(description="Name of location")
    description: str = Field(description="Description/role of location in this story")

class LegendLocationList(BaseModel):
    """List of locations."""
    locations: list[LegendLocation] = Field(description="List of locations. Only include locations from the original list of locations.")

legendloc_llm = llm.with_structured_output(LegendLocationList)

In [113]:
# get character details
char_prompt = f"list of characters: {characters}"
invalid_input = True
while invalid_input:
    try:
        res = legendchar_llm.invoke(char_prompt)
        invalid_input = False
    except Exception as e:
        print("Exception thrown. Trying again.")
        print("Error:", e)
        invalid_input = True

In [114]:
character_info = []
for character in res.characters:
    character_info.append({"name": character.name, "description": character.description, "group": character.group})

len(character_info), character_info

(18,
 [{'name': 'Lila',
   'description': 'Protagonist and aspiring artist',
   'group': 'main characters'},
  {'name': 'Passersby',
   'description': 'Background characters illustrating the dullness of the world',
   'group': 'background characters'},
  {'name': 'Zephyr',
   'description': 'Mischievous spirit guiding Lila',
   'group': 'main characters'},
  {'name': 'Red Keeper',
   'description': 'Guardian of the color red',
   'group': 'Color Keepers'},
  {'name': 'Blue Keeper',
   'description': 'Guardian of the color blue',
   'group': 'Color Keepers'},
  {'name': 'Yellow Keeper',
   'description': 'Guardian of the color yellow',
   'group': 'Color Keepers'},
  {'name': 'Green Keeper',
   'description': 'Guardian of the color green',
   'group': 'Color Keepers'},
  {'name': 'Malachai',
   'description': 'The dark sorcerer revealing his intentions to drain color from the world.',
   'group': 'antagonists'},
  {'name': 'Sage',
   'description': 'The wise sage providing guidance',
  

In [115]:
# save the character details to a file
with open(f"json/{story_name}/characters.json", "w") as f:
    json.dump(character_info, f, indent=4)

and location details

In [116]:
# get location details
loc_prompt = f"list of locations: {locations}. story info: {scenes}"
invalid_input = True
while invalid_input:
    try:
        res = legendloc_llm.invoke(loc_prompt)
        invalid_input = False
    except Exception as e:
        print("Exception thrown. Trying again.")
        print("Error:", e)
        invalid_input = True

In [117]:
location_info = []
for location in res.locations:
    location_info.append({"name": location.name, "description": location.description})

len(location_info), location_info

(12,
 [{'name': "Lila's House",
   'description': 'Lila wakes up in her gray bedroom, surrounded by monochrome art supplies.'},
  {'name': 'Monochrome Town',
   'description': 'Lila walks through her dull town, observing the lack of color in her surroundings.'},
  {'name': 'Art Studio',
   'description': 'Lila attempts to paint but feels uninspired and blocked by her gray surroundings.'},
  {'name': "Lila's Dreamscape",
   'description': 'Lila catches a fleeting glimpse of color in her dreams, igniting her desire to find it.'},
  {'name': 'Abandoned Art Gallery',
   'description': 'Lila and Zephyr stumble upon an ancient mural that reveals the existence of the hidden realm.'},
  {'name': 'Hidden Realm Of Colors',
   'description': 'Lila and Zephyr step through the portal, entering the hidden realm of colors.'},
  {'name': "Color Keepers' Sanctuary",
   'description': 'Lila enters the sanctuary and is awed by the vibrant colors surrounding her.'},
  {'name': "Malachai's Lair",
   'descr

In [118]:
# save the location details to a file
with open(f"json/{story_name}/locations.json", "w") as f:
    json.dump(location_info, f, indent=4)

## save complete story outline

In [119]:
# compile all the information into a single json file

# read in idea
with open(f"json/{story_name}/idea.json") as f:
    idea = json.load(f)

# read in chapters
with open(f"json/{story_name}/chapters.json") as f:
   chapters = json.load(f)

# read in scenes
with open(f"json/{story_name}/scenes.json") as f:
    scenes = json.load(f)

# read in characters
with open(f"json/{story_name}/characters.json") as f:
    characters = json.load(f)

# read in locations
with open(f"json/{story_name}/locations.json") as f:
    locations = json.load(f)

story_outline = {
    "title": idea["title"],
    "type": "Book",
    "author": "ChatGPT",
    "year": 2024,
    "num_chapters": len(chapters),
    "num_scenes": len(scenes),
    "num_characters": len(characters),
    "num_locations": len(locations),
    "chapters": chapters,
    "scenes": scenes,
    "characters": characters,
    "locations": locations   
}

story_outline

{'title': 'The Color Thief',
 'type': 'Book',
 'author': 'ChatGPT',
 'year': 2024,
 'num_chapters': 12,
 'num_scenes': 48,
 'num_characters': 18,
 'num_locations': 12,
 'chapters': [{'chapter': 'Chapter 1: The Gray Awakening',
   'summary': 'In a world devoid of color, a young artist named Lila struggles to find inspiration.',
   'description': 'Lila wakes up in her monochrome town, feeling the weight of the dullness around her. She reflects on her dreams of color and beauty, but her surroundings stifle her creativity. This chapter introduces Lila and her longing for a vibrant world, setting the stage for her journey.',
   'importance': 0.9,
   'conflict': 0.3},
  {'chapter': 'Chapter 2: The Mysterious Encounter',
   'summary': 'Lila meets a mischievous spirit named Zephyr who reveals the existence of a hidden realm.',
   'description': "While wandering through the gray streets, Lila encounters Zephyr, a playful spirit who can manipulate shadows. Zephyr tells her about a hidden realm w

In [120]:
# save the story outline to a file
with open(f"json/{story_name}/story_outline.json", "w") as f:
    json.dump(story_outline, f, indent=4)

In [121]:
# check if any locations are missing from scenes

# Load the JSON data from a file
with open(f"json/{story_name}/story_outline.json", "r") as file:
    data = json.load(file)

# Extract the locations from the locations object
all_locations = {location["name"] for location in data["locations"]}

# Extract the locations used in scenes
scene_locations = {scene["location"] for scene in data["scenes"]}

# Check if each location in "locations" is included in "scenes"
missing_locations = all_locations - scene_locations

# also other way around
extra_locations = scene_locations - all_locations

# Display the result
if missing_locations:
    print("The following locations are not included in any scene:")
    for location in missing_locations:
        print(f"- {location}")
else:
    print("All locations are included in at least one scene.")

if extra_locations:
    print("The following locations are not defined but used in scenes:")
    for location in extra_locations:
        print(f"- {location}")
else:
    print("All locations used in scenes are defined.")

All locations are included in at least one scene.
All locations used in scenes are defined.


In [122]:
# check characters too
all_characters = {character["name"] for character in data["characters"]}
scene_characters = {character["name"] for scene in data["scenes"] for character in scene["characters"]}
missing_characters = all_characters - scene_characters
extra_characters = scene_characters - all_characters

if missing_characters:
    print("\nThe following characters are not included in any scene:")
    for character in missing_characters:
        print(f"- {character}")

if extra_characters:
    print("\nThe following characters are not defined but used in scenes:")
    for character in extra_characters:
        print(f"- {character}")

In [123]:
# count characters/locations in scenes
with open(f'json/{story_name}/story_outline.json', 'r') as file:
    story_outline = json.load(file)

scenes = story_outline['scenes']
chapters = story_outline['chapters']
characters = story_outline['characters']
locations = story_outline['locations']

In [124]:
# count how many scenes each character is in per chapter
character_scene_count = {}

for scene in scenes:
    chapter = scene['chapter']  # Ensure this is a string or integer
    if chapter not in character_scene_count:
        character_scene_count[chapter] = {}
        
    for character in scene['characters']:
        character_name = character['name']  # Extract a unique identifier from the character dictionary
        if character_name not in character_scene_count[chapter]:
            character_scene_count[chapter][character_name] = 1
        else:
            character_scene_count[chapter][character_name] += 1

character_scene_count

{'Chapter 1: The Gray Awakening': {'Lila': 5},
 'Chapter 2: The Mysterious Encounter': {'Lila': 1, 'Passersby': 1},
 'Chapter 3: The Portal to Color': {'Lila': 4, 'Zephyr': 4},
 'Chapter 4: The Color Keepers': {'Lila': 3,
  'Zephyr': 1,
  'Red Keeper': 1,
  'Blue Keeper': 1,
  'Yellow Keeper': 1,
  'Green Keeper': 1},
 "Chapter 5: The Thief's Lair": {'Lila': 5, 'Zephyr': 5, 'Malachai': 2},
 'Chapter 6: Allies and Enemies': {'Lila': 1,
  'Sage': 1,
  'Finn': 1,
  'Brave Warrior': 1},
 'Chapter 7: The Battle for Color': {'Lila': 8,
  'Zephyr': 7,
  'Allies': 6,
  "Malachai's Minions": 1,
  'Malachai': 2},
 'Chapter 8: The Heart of the Realm': {'Lila': 4,
  'Zephyr': 4,
  'Allies': 4,
  'Guardian Of Colors': 2},
 'Chapter 9: Betrayal and Sacrifice': {'Lila': 1, 'Milo': 1, 'Sera': 1},
 'Chapter 10: The Final Confrontation': {'Lila': 7,
  'Zephyr': 2,
  'Malachai': 5},
 'Chapter 11: Restoration of Color': {'Lila': 5,
  'Zephyr': 5,
  'Color Keepers': 5},
 'Chapter 12: A New Beginning': {'Li

In [125]:
# repeat for locations
location_scene_count = {}

for scene in scenes:
    chapter = scene['chapter']
    if chapter not in location_scene_count:
        location_scene_count[chapter] = {}
        
    location_name = scene['location']
    if location_name not in location_scene_count[chapter]:
        location_scene_count[chapter][location_name] = 1
    else:
        location_scene_count[chapter][location_name] += 1

location_scene_count

{'Chapter 1: The Gray Awakening': {"Lila's House": 1,
  'Monochrome Town': 2,
  'Art Studio': 1,
  "Lila's Dreamscape": 1},
 'Chapter 2: The Mysterious Encounter': {'Monochrome Town': 1},
 'Chapter 3: The Portal to Color': {'Abandoned Art Gallery': 2,
  'Hidden Realm Of Colors': 2},
 'Chapter 4: The Color Keepers': {"Color Keepers' Sanctuary": 3},
 "Chapter 5: The Thief's Lair": {"Malachai's Lair": 5},
 'Chapter 6: Allies and Enemies': {'Vibrant Glade': 1},
 'Chapter 7: The Battle for Color': {'Hidden Realm Of Colors': 1,
  'The Battlefield': 7},
 'Chapter 8: The Heart of the Realm': {'Hidden Realm Of Colors': 4},
 'Chapter 9: Betrayal and Sacrifice': {'The Hidden Base Of The Group': 1},
 'Chapter 10: The Final Confrontation': {"Lila's House": 1,
  "Malachai's Lair": 5,
  'Monochrome Town': 1},
 'Chapter 11: Restoration of Color': {'The Battlefield': 1,
  'Hidden Realm Of Colors': 2,
  'The Once Monochrome Landscape Now Bursting With Color': 1,
  'Art Studio': 1},
 'Chapter 12: A New B

In [126]:
# update counts in chapter json object
for chapter in chapters:
    chap_name = chapter['chapter']
    print(chap_name)

    # add character counts
    chap_characters = character_scene_count[chap_name]
    new_char_dict = {}
    for char in chap_characters:
        # find char obj in characters with matching name
        char_obj = [c for c in characters if c["name"] == char][0]
        new_char_dict[char] = {"count": chap_characters[char], "role": char_obj["description"]}
   
    # add location counts
    chap_locations = location_scene_count[chap_name]
    new_loc_dict = {}
    for loc in chap_locations:
        # find loc obj in locations with matching name
        loc_obj = [l for l in locations if l["name"] == loc][0]
        new_loc_dict[loc] = {"count": chap_locations[loc], "role": loc_obj["description"]}

    chapter['characters'] = new_char_dict
    chapter['locations'] = new_loc_dict

chapters

Chapter 1: The Gray Awakening
Chapter 2: The Mysterious Encounter
Chapter 3: The Portal to Color
Chapter 4: The Color Keepers
Chapter 5: The Thief's Lair
Chapter 6: Allies and Enemies
Chapter 7: The Battle for Color
Chapter 8: The Heart of the Realm
Chapter 9: Betrayal and Sacrifice
Chapter 10: The Final Confrontation
Chapter 11: Restoration of Color
Chapter 12: A New Beginning


[{'chapter': 'Chapter 1: The Gray Awakening',
  'summary': 'In a world devoid of color, a young artist named Lila struggles to find inspiration.',
  'description': 'Lila wakes up in her monochrome town, feeling the weight of the dullness around her. She reflects on her dreams of color and beauty, but her surroundings stifle her creativity. This chapter introduces Lila and her longing for a vibrant world, setting the stage for her journey.',
  'importance': 0.9,
  'conflict': 0.3,
  'characters': {'Lila': {'count': 5,
    'role': 'Protagonist and aspiring artist'}},
  'locations': {"Lila's House": {'count': 1,
    'role': 'Lila wakes up in her gray bedroom, surrounded by monochrome art supplies.'},
   'Monochrome Town': {'count': 2,
    'role': 'Lila walks through her dull town, observing the lack of color in her surroundings.'},
   'Art Studio': {'count': 1,
    'role': 'Lila attempts to paint but feels uninspired and blocked by her gray surroundings.'},
   "Lila's Dreamscape": {'count

In [127]:
# save the updated story outline to a file
with open(f"json/{story_name}/story_outline.json", "w") as f:
    json.dump(story_outline, f, indent=4)

In [128]:
# rank by importance and conflict
with open(f'json/{story_name}/story_outline.json', 'r') as file:
    story_outline = json.load(file)

scenes = story_outline['scenes']
chapters = story_outline['chapters']
characters = story_outline['characters']
locations = story_outline['locations']

In [129]:
# rank each scene by importance
# and within each scene, the characters

for i, chapter in enumerate(chapters):
    importances = []
    conflicts = []
    num_scenes = 0

    chap_name = chapter["chapter"]
    
    # extract importance from each scene
    for j, scene in enumerate(scenes):
        if scene["chapter"] != chap_name:
            continue
        importances.append((j, scene["importance"]))
        conflicts.append((j, scene["conflict"]))
        # now extract character importances
        character_importances = []
        for k, character in enumerate(scene["characters"]):
            character_importances.append((k, character["importance"]))
        # sort character importances
        sorted_character_importances = sorted(character_importances, key=lambda x: x[1], reverse=True)
        # add importance_rank to each character
        for k, (l, _) in enumerate(sorted_character_importances):
            scenes[j]["characters"][l]["importance_rank"] = k+1
        # add number of each scene
        scenes[j]["number"] = j+1
        num_scenes += 1
    # sort importances
    sorted_importances = sorted(importances, key=lambda x: x[1], reverse=True)
    # add importance_rank to each scene
    for k, (j, _) in enumerate(sorted_importances):
        scenes[j]["importance_rank"] = k+1
    # sort conflicts
    sorted_conflicts = sorted(conflicts, key=lambda x: x[1], reverse=True)
    # add conflict_rank to each scene
    for k, (j, _) in enumerate(sorted_conflicts):
        scenes[j]["conflict_rank"] = k+1
    
    # add number of scenes to chapter
    chapters[i]["scenes"] = num_scenes
    
    # print results
    # for j, scene in enumerate(scenes):
    #     print(scenes[j]["title"], scenes[j]["importance_rank"])
    #     print("--------------------------------")
    #     for k, character in enumerate(scene["characters"]):
    #         print(character["name"], character["importance_rank"])
    #     print()

In [130]:
scenes

[{'title': "Lila's Awakening",
  'number': 1,
  'summary': 'Lila wakes up in her gray bedroom, surrounded by monochrome art supplies.',
  'chapter': 'Chapter 1: The Gray Awakening',
  'location': "Lila's House",
  'importance': 0.9,
  'conflict': 0.2,
  'characters': [{'name': 'Lila',
    'role': 'Protagonist and aspiring artist',
    'importance': 1.0,
    'emotion': 'Frustrated and longing',
    'sentiment': -0.5,
    'importance_rank': 1}],
  'importance_rank': 1,
  'conflict_rank': 4},
 {'title': 'The Gray Town',
  'number': 2,
  'summary': 'Lila walks through her dull town, observing the lack of color in her surroundings.',
  'chapter': 'Chapter 1: The Gray Awakening',
  'location': 'Monochrome Town',
  'importance': 0.8,
  'conflict': 0.3,
  'characters': [{'name': 'Lila',
    'role': 'Protagonist reflecting on her environment',
    'importance': 1.0,
    'emotion': 'Melancholic',
    'sentiment': -0.6,
    'importance_rank': 1}],
  'importance_rank': 3,
  'conflict_rank': 3},
 {

In [131]:
chapters

[{'chapter': 'Chapter 1: The Gray Awakening',
  'summary': 'In a world devoid of color, a young artist named Lila struggles to find inspiration.',
  'description': 'Lila wakes up in her monochrome town, feeling the weight of the dullness around her. She reflects on her dreams of color and beauty, but her surroundings stifle her creativity. This chapter introduces Lila and her longing for a vibrant world, setting the stage for her journey.',
  'importance': 0.9,
  'conflict': 0.3,
  'characters': {'Lila': {'count': 5,
    'role': 'Protagonist and aspiring artist'}},
  'locations': {"Lila's House": {'count': 1,
    'role': 'Lila wakes up in her gray bedroom, surrounded by monochrome art supplies.'},
   'Monochrome Town': {'count': 2,
    'role': 'Lila walks through her dull town, observing the lack of color in her surroundings.'},
   'Art Studio': {'count': 1,
    'role': 'Lila attempts to paint but feels uninspired and blocked by her gray surroundings.'},
   "Lila's Dreamscape": {'count

In [132]:
# save the updated story outline to a file
with open(f"json/{story_name}/story_outline.json", "w") as f:
    json.dump(story_outline, f, indent=4)

## generate text

In [133]:
# read in story outline
with open(f"json/{story_name}/story_outline.json") as f:
    story_outline = json.load(f)

title = story_outline["title"]
chapters = story_outline["chapters"]
scenes = story_outline["scenes"]
chapter_names = [chapter["chapter"] for chapter in chapters]

In [134]:
# create a folder to store the chapter files
if not os.path.exists(f"json/{story_name}/chapters"):
    os.makedirs(f"json/{story_name}/chapters")

In [ ]:
for i, chapter in enumerate(chapter_names):
    chap_scenes = [scene for scene in scenes if scene["chapter"] == chapter]
    num_scenes = len(chap_scenes)
    print(f"{chapter}")
    print(f"Number of scenes: {num_scenes}")
    prompt = f"""You are a novelist. Using the story outline below, please write out {chapter}, 
                delineating each scene with "XXX" and making sure each scene has exactly the correct 
                characters (and their respective emotions, importance, etc.), location, and using the 
                chapter and scene summaries to guide your writing. 
                All (and no other) characters and locations should be used.

                Don't explicitly write the scene titles or locations in your output. Each scene should 
                be at least 10 paragraphs long. This chapter should have exactly {num_scenes} scene{'s' if num_scenes > 1 else ''}.

                Here is the story outline:
                {chap_scenes}

                Just output the chapter text (with "XXX" dividers between each scene) and nothing else. 
                Don't add "XXX" at the end of the last scene.
                """
    
    invalid_input = True
    while invalid_input: # try until valid input
        # see if exception is thrown
        try:
            res = llm.invoke(prompt)
            invalid_input = False
        except Exception as e:
            print("Exception thrown. Trying again.")
            print("Error:", e)
            invalid_input = True

    # save the chapter text to a file
    with open(f"json/{story_name}/chapters/{chapter}.txt", "w") as f:
        f.write(res.content)

    print(f"{chapter} written.")

Chapter 7: The Battle for Color
Number of scenes: 8
Chapter 7: The Battle for Color written.


In [152]:
# check number of scenes in each chapter
for i, chapter in enumerate(chapters):
    # read in chapter text
    with open(f"json/{story_name}/chapters/{chapter['chapter']}.txt") as f:
        chapter_text = f.read()
    
    # count number of scenes (number of 'XXX' dividers + 1)
    num_scenes = chapter_text.count("XXX") + 1

    # compare to expected number of scenes
    if num_scenes == chapter["scenes"]:
        print(f"{chapter['chapter']} has the correct number of scenes.")
    else:
        print(f"{chapter['chapter']} has the wrong number of scenes. Expected {chapter['scenes']}, got {num_scenes}.")

Chapter 1: The Gray Awakening has the correct number of scenes.
Chapter 2: The Mysterious Encounter has the correct number of scenes.
Chapter 3: The Portal to Color has the correct number of scenes.
Chapter 4: The Color Keepers has the correct number of scenes.
Chapter 5: The Thief's Lair has the correct number of scenes.
Chapter 6: Allies and Enemies has the correct number of scenes.
Chapter 7: The Battle for Color has the correct number of scenes.
Chapter 8: The Heart of the Realm has the correct number of scenes.
Chapter 9: Betrayal and Sacrifice has the correct number of scenes.
Chapter 10: The Final Confrontation has the correct number of scenes.
Chapter 11: Restoration of Color has the correct number of scenes.
Chapter 12: A New Beginning has the correct number of scenes.


In [153]:
# combine all chapters into a single file
chapter_files = [f"json/{story_name}/chapters/{chapter['chapter']}.txt" for chapter in chapters]

# read in all chapter files
chapter_texts = []
for file in chapter_files:
    with open(file) as f:
        chapter_texts.append(f.read())

chapter_texts

["Lila stirred awake, the familiar grayness of her bedroom wrapping around her like a heavy blanket. The walls, painted in a muted shade that matched the overcast sky outside, seemed to absorb the light rather than reflect it. Her gaze drifted to the scattered art supplies that littered her desk—brushes, charcoal, and canvases—all in shades of gray. It was as if the world had conspired to drain color from her life, leaving her with nothing but a palette of shadows. Frustration bubbled within her, a longing for something vibrant, something that could break through the monotony of her existence.\n\nShe swung her legs over the side of the bed, her feet touching the cold wooden floor. The chill sent a shiver up her spine, but it was nothing compared to the chill that settled in her heart. Lila had always dreamed of being an artist, of creating pieces that would evoke emotion and stir the soul. Yet here she was, surrounded by supplies that felt more like shackles than tools of expression. T

In [154]:
# add chapter title to beginning of each chapter
combined_text = f"{title}\n\n"

for i, chapter in enumerate(chapters):
    combined_text += f"{chapter['chapter']}\n\n"
    combined_text += chapter_texts[i]
    combined_text += "\n\n"

combined_text

'The Color Thief\n\nChapter 1: The Gray Awakening\n\nLila stirred awake, the familiar grayness of her bedroom wrapping around her like a heavy blanket. The walls, painted in a muted shade that matched the overcast sky outside, seemed to absorb the light rather than reflect it. Her gaze drifted to the scattered art supplies that littered her desk—brushes, charcoal, and canvases—all in shades of gray. It was as if the world had conspired to drain color from her life, leaving her with nothing but a palette of shadows. Frustration bubbled within her, a longing for something vibrant, something that could break through the monotony of her existence.\n\nShe swung her legs over the side of the bed, her feet touching the cold wooden floor. The chill sent a shiver up her spine, but it was nothing compared to the chill that settled in her heart. Lila had always dreamed of being an artist, of creating pieces that would evoke emotion and stir the soul. Yet here she was, surrounded by supplies that 

In [155]:
# save the combined text to a file
with open(f"scripts/{story_name}.txt", "w") as f:
    f.write(combined_text)

## format story

In [156]:
# read in combined text
with open(f"scripts/{story_name}.txt") as f:
    story_text = f.read()

In [157]:
# replace curly quotes with straight quotes
story_text = story_text.replace("“", "\"").replace("”", "\"").replace("‘", "'").replace("’", "'")

In [158]:
# save the combined text to a file
with open(f"scripts/{story_name}.txt", "w") as f:
    f.write(story_text)

In [159]:
def split_long_lines(filename, folder="scripts", max_length=80):
    # Open the file for reading
    with open(folder + "/" + filename, 'r') as file:
        lines = file.readlines()

    new_lines = []
    
    # Iterate through each line
    for line in lines:
        while len(line) > max_length:
            # Find the last space before the max_length
            split_index = line[:max_length].rfind(' ')
            if split_index == -1:
                split_index = max_length  # No space found, split at max_length
                
            # Append the portion of the line up to the split point
            new_lines.append(line[:split_index].strip() + '\n')
            
            # Continue processing the rest of the line
            line = line[split_index:].strip()
        
        new_lines.append(line + '\n')  # Add the rest of the line (if any) or short lines directly

    # remove any lines that are 'XXX'
    new_lines = [line for line in new_lines if line.strip() != 'XXX']
    
    # Write the modified lines back to the file or a new file
    with open(folder + '/split_' + filename, 'w') as file:
        file.writelines(new_lines)

In [160]:
split_long_lines(f'{story_name}.txt')

In [161]:
def remove_extra_newlines(filename, folder):
    # Open the file for reading
    with open(folder + "/" + filename, 'r') as file:
        lines = file.read()
    
    # Replace 3+ newlines with just 2 newlines
    cleaned_lines = re.sub(r'\n{3,}', '\n\n', lines)

    # Write the cleaned content back to the file (or a new file)
    with open(folder + "/" + filename, 'w') as file:
        file.write(cleaned_lines)

In [162]:
remove_extra_newlines(f'split_{story_name}.txt', 'scripts')

In [163]:
# rename files
# story_name.txt --> story_name_og.txt
# split_story_name.txt --> story_name.txt

os.rename(f"scripts/{story_name}.txt", f"scripts/{story_name}_og.txt")
os.rename(f"scripts/split_{story_name}.txt", f"scripts/{story_name}.txt")

## get line numbers for each scene (after parsing-data.ipynb is run)

In [164]:
def get_scene_boundaries(text_file_path, line_file_path):
    # Read the main text file and find positions of "XXX"
    with open(text_file_path, 'r') as text_file:
        text_lines = text_file.readlines()
    
    # read file with line numbers
    with open(line_file_path, 'r') as line_file:
        line_numbers = line_file.readlines()
    
    # Gather start and end line numbers for each scene based on "XXX" markers
    scene_boundaries = []
    scene_start = 1  # Assuming line numbers start from 1
    for idx, line in enumerate(text_lines):
        if "XXX" in line:
            # find first line after "XXX" that is not empty
            next_line = text_lines[idx + 2][:50]

            # find line number in line_numbers file
            end_index = -1
            for i, line_num in enumerate(line_numbers):
                if next_line in line_num:
                    end_index = i + 1
                    scene_boundaries.append((scene_start, end_index - 1))
                    scene_start = end_index  # Set next scene start after "XXX"
                    idx += 3
                    break
    
    # add last scene boundary
    scene_boundaries.append((scene_start, len(line_numbers)))
    
    
    # Translate scene boundaries to line numbers
    # scene_line_numbers = [(line_numbers[start - 1], line_numbers[end - 1]) for start, end in scene_boundaries]
    
    return scene_boundaries

In [81]:
# Paths to your files
chapter_folder = f'chapters/{story_name}'              # Folder with chapter files
scripts_folder = f'scripts'                         # Folder with script files
text_file_path = f'{scripts_folder}/{story_name}_og.txt'  # File with main text and "XXX"

# read in story_outline.json
with open(f'json/{story_name}/story_outline.json', 'r') as file:
    story_outline = json.load(file)

chapters = story_outline['chapters']
scenes = story_outline['scenes']

chapter_lines = {}

for chapter in chapters:
    chapter_name = chapter['chapter']
    line_file_path = f'{chapter_folder}/{chapter_name}.txt'      # File with line numbers

    # Call the function and display results
    scene_line_numbers = get_scene_boundaries(text_file_path, line_file_path) 

    # Print scene line numbers
    print(chapter_name)
    for idx, (start, end) in enumerate(scene_line_numbers, 1):
        print(f"scene {idx}: starts at line {start}, ends at line {end}")
    print()

    chapter_lines[chapter_name] = scene_line_numbers

Chapter 1: The Dusty Shelves
scene 1: starts at line 1, ends at line 46
scene 2: starts at line 47, ends at line 76
scene 3: starts at line 77, ends at line 101
scene 4: starts at line 102, ends at line 127
scene 5: starts at line 128, ends at line 159

Chapter 2: Whispers of the Past
scene 1: starts at line 1, ends at line 71

Chapter 3: The Town's Dreamers
scene 1: starts at line 1, ends at line 46
scene 2: starts at line 47, ends at line 83
scene 3: starts at line 84, ends at line 118
scene 4: starts at line 119, ends at line 159

Chapter 4: The Artist's Struggle
scene 1: starts at line 1, ends at line 73

Chapter 5: The Teacher's Regret
scene 1: starts at line 1, ends at line 68

Chapter 6: The Single Mother's Dilemma
scene 1: starts at line 1, ends at line 29
scene 2: starts at line 30, ends at line 56
scene 3: starts at line 57, ends at line 81
scene 4: starts at line 82, ends at line 112
scene 5: starts at line 113, ends at line 137

Chapter 7: The Town Meeting
scene 1: starts a

In [82]:
chapter_lines

{'Chapter 1: The Dusty Shelves': [(1, 46),
  (47, 76),
  (77, 101),
  (102, 127),
  (128, 159)],
 'Chapter 2: Whispers of the Past': [(1, 71)],
 "Chapter 3: The Town's Dreamers": [(1, 46), (47, 83), (84, 118), (119, 159)],
 "Chapter 4: The Artist's Struggle": [(1, 73)],
 "Chapter 5: The Teacher's Regret": [(1, 68)],
 "Chapter 6: The Single Mother's Dilemma": [(1, 29),
  (30, 56),
  (57, 81),
  (82, 112),
  (113, 137)],
 'Chapter 7: The Town Meeting': [(1, 70)],
 'Chapter 8: The Rival Bookstore': [(1, 74)],
 'Chapter 9: The Turning Point': [(1, 27),
  (28, 50),
  (51, 75),
  (76, 99),
  (100, 124)],
 'Chapter 10: The Dream Festival': [(1, 34),
  (35, 72),
  (73, 106),
  (107, 139),
  (140, 176),
  (177, 216)],
 'Chapter 11: Confronting Fears': [(1, 60)],
 'Chapter 12: The New Chapter': [(1, 39),
  (40, 73),
  (74, 111),
  (112, 146),
  (147, 177)]}

In [83]:
# add "first_line" and "last_line" to each scene
for chapter in chapters:
    chapter_name = chapter['chapter']
    print(chapter_name)
    scene_line_numbers = chapter_lines[chapter_name]
    print(scene_line_numbers)
    scene_num = 0
    for i, scene in enumerate(scenes):
        compare_chapter = chapter_name
        if scene['chapter'] == compare_chapter:
            print(scene['chapter'], compare_chapter)
            print(scene_num)
            scene['first_line'] = scene_line_numbers[scene_num][0] 
            scene['last_line'] = scene_line_numbers[scene_num][1]
            scene_num += 1

Chapter 1: The Dusty Shelves
[(1, 46), (47, 76), (77, 101), (102, 127), (128, 159)]
Chapter 1: The Dusty Shelves Chapter 1: The Dusty Shelves
0
Chapter 1: The Dusty Shelves Chapter 1: The Dusty Shelves
1
Chapter 1: The Dusty Shelves Chapter 1: The Dusty Shelves
2
Chapter 1: The Dusty Shelves Chapter 1: The Dusty Shelves
3
Chapter 1: The Dusty Shelves Chapter 1: The Dusty Shelves
4
Chapter 2: Whispers of the Past
[(1, 71)]
Chapter 2: Whispers of the Past Chapter 2: Whispers of the Past
0
Chapter 3: The Town's Dreamers
[(1, 46), (47, 83), (84, 118), (119, 159)]
Chapter 3: The Town's Dreamers Chapter 3: The Town's Dreamers
0
Chapter 3: The Town's Dreamers Chapter 3: The Town's Dreamers
1
Chapter 3: The Town's Dreamers Chapter 3: The Town's Dreamers
2
Chapter 3: The Town's Dreamers Chapter 3: The Town's Dreamers
3
Chapter 4: The Artist's Struggle
[(1, 73)]
Chapter 4: The Artist's Struggle Chapter 4: The Artist's Struggle
0
Chapter 5: The Teacher's Regret
[(1, 68)]
Chapter 5: The Teacher's 

In [84]:
scenes

[{'title': 'The Opening of the Bookstore',
  'number': 1,
  'summary': 'Emma opens the bookstore for the day, reflecting on her struggles.',
  'chapter': 'Chapter 1: The Dusty Shelves',
  'location': 'The Bookstore Of Forgotten Dreams',
  'importance': 0.8,
  'conflict': 0.2,
  'characters': [{'name': 'Emma',
    'role': 'Owner of the bookstore, passionate about literature',
    'importance': 1.0,
    'emotion': 'Determined yet anxious',
    'sentiment': 0.0,
    'importance_rank': 1},
   {'name': 'Mr. Thompson',
    'role': 'Regular customer, offers support',
    'importance': 0.5,
    'emotion': 'Sympathetic',
    'sentiment': 1.0,
    'importance_rank': 2}],
  'importance_rank': 3,
  'conflict_rank': 4,
  'first_line': 1,
  'last_line': 46},
 {'title': 'A Walk Through the Aisles',
  'number': 2,
  'summary': 'Emma walks through the aisles, reminiscing about her dreams.',
  'chapter': 'Chapter 1: The Dusty Shelves',
  'location': 'The Bookstore Of Forgotten Dreams',
  'importance': 0

In [85]:
# update json file
with open(f'json/{story_name}/story_outline.json', 'w') as file:
    json.dump(story_outline, file, indent=4)